Iniciando bibliotecas e salvando informacoes

In [1]:
import zipfile
import os
import pandas as pd

arquivo_zip = "data/HIST_PAINEL_COVIDBR_18nov2020.zip"
arquivo_csv = "data/HIST_PAINEL_COVIDBR_18nov2020.csv"

In [2]:

# Extração do arquivo zip
def extrair_dados_zip(zip):
  print("Extraindo arquivo ZIP...")
  data = zipfile.ZipFile(zip, "r")
  data = data.extractall("data")
  
# Ler dados da planilha
def ler_dados_csv(csv):
  print("Lendo a planilha...")
  data = pd.read_csv(csv, encoding="utf-8",delimiter=";")
  return data

# Chamando função caso não tenha extraido
if not os.path.exists(f"{arquivo_csv}"):
  extrair_dados_zip(f"{arquivo_zip}")

# Lendo planilha
data = ler_dados_csv(arquivo_csv)

Lendo a planilha...


In [3]:
print(data.iloc[55])

regiao                         Brasil
estado                            NaN
municipio                         NaN
coduf                              76
codmun                            NaN
codRegiaoSaude                    NaN
nomeRegiaoSaude                   NaN
data                       2020-04-20
semanaEpi                          17
populacaoTCU2019          210147125.0
casosAcumulado                  40581
casosNovos                       1927
obitosAcumulado                  2575
obitosNovos                       113
Recuperadosnovos              22991.0
emAcompanhamentoNovos         15015.0
interior/metropolitana            NaN
Name: 55, dtype: object


In [4]:
def relatorio_meses(data, eh_estado, relatorio_brasil):
  lista_meses = []
  casos_acumulados = 0
  obitos_acumulados = 0

  for mes in range(2,12):
    data_mes = data[data["data"].dt.month == mes]
    casos = data_mes["casosNovos"].sum()
    obitos = data_mes["obitosNovos"].sum()
    populacao = data.iloc[0]["populacaoTCU2019"]
    if not eh_estado:
      infectados = data_mes.iloc[-1]["emAcompanhamentoNovos"]
    
    casos_acumulados += casos
    obitos_acumulados += obitos
    
    mes = int(mes)
    variacao_casos = 0
    variacao_obitos = 0
    
    if mes > 2:
      casos_antigo = lista_meses[mes-3][1]
      obitos_antigo = lista_meses[mes-3][2]
      
      variacao_casos =  (casos / casos_antigo) - 1 if casos_antigo != 0 else casos
      variacao_casos *= 100
      variacao_casos = round(variacao_casos, 2)
      
      variacao_obitos = (obitos / obitos_antigo) - 1 if obitos_antigo != 0 else obitos
      variacao_obitos *= 100
      variacao_obitos = round(variacao_obitos, 2)
      
    casos_por_100_mil = (casos / populacao) * 100000
    casos_por_100_mil = round(casos_por_100_mil, 2)
    obitos_por_100_mil = (obitos / populacao) * 100000
    obitos_por_100_mil = round(obitos_por_100_mil, 2)
    
    colunas = ["mes", "casosNovos", "obitosNovos", "variacaoDeCasos" ,"variacaoDeObitos", "casosAcumulados", "obitosAcumulados", "Casos por 100 mil habitantes", "Obitos por 100 mil habitantes"]
    dados_colunas = [mes, casos, obitos, variacao_casos, variacao_obitos, casos_acumulados, obitos_acumulados, casos_por_100_mil, obitos_por_100_mil]
    if eh_estado:
      colunas += "Comparação Nacional (Casos)", "Comparação Nacional (Obitos)"
      relatorio_mes = relatorio_brasil[relatorio_brasil.index == mes]
      comparacao_nacional_casos = casos / relatorio_mes["casosNovos"].sum()
      comparacao_nacional_casos *=  100
      comparacao_nacional_casos = round(comparacao_nacional_casos, 2) 
      
      comparacao_nacional_obitos = obitos / relatorio_mes["obitosNovos"].sum()
      comparacao_nacional_obitos *=  100
      comparacao_nacional_obitos = round(comparacao_nacional_obitos, 2) 
      
      dados_colunas += comparacao_nacional_casos, comparacao_nacional_obitos
      
    else:
      colunas.insert(4, "infectados")
      dados_colunas.insert(4, infectados)
    
    lista_meses.append(dados_colunas)

  data_meses = pd.DataFrame(lista_meses, columns=colunas)
  data_meses = data_meses.set_index("mes")
  return data_meses

## Retirando dados sobre o brasil

In [5]:
data["data"] = pd.to_datetime(data["data"], format="%Y-%m-%d")

print(data.dtypes)

regiao                            object
estado                            object
municipio                         object
coduf                              int64
codmun                           float64
codRegiaoSaude                   float64
nomeRegiaoSaude                   object
data                      datetime64[ns]
semanaEpi                          int64
populacaoTCU2019                 float64
casosAcumulado                     int64
casosNovos                         int64
obitosAcumulado                    int64
obitosNovos                        int64
Recuperadosnovos                 float64
emAcompanhamentoNovos            float64
interior/metropolitana           float64
dtype: object


In [6]:
# Dados do brasil
data_brasil = data[data["regiao"] == "Brasil"]

In [7]:
print(data_brasil.iloc[0])

regiao                                 Brasil
estado                                    NaN
municipio                                 NaN
coduf                                      76
codmun                                    NaN
codRegiaoSaude                            NaN
nomeRegiaoSaude                           NaN
data                      2020-02-25 00:00:00
semanaEpi                                   9
populacaoTCU2019                  210147125.0
casosAcumulado                              0
casosNovos                                  0
obitosAcumulado                             0
obitosNovos                                 0
Recuperadosnovos                          NaN
emAcompanhamentoNovos                     NaN
interior/metropolitana                    NaN
Name: 0, dtype: object


In [8]:
# A maior quantidade de novos casos no Brasil
maximo_casos_novos_brasil = data_brasil["casosNovos"].max()
data_casos_brasil = data_brasil[data_brasil["casosNovos"] == maximo_casos_novos_brasil]
maximo_casos_brasil = data_casos_brasil["casosNovos"]
date_casos_brasil = data_casos_brasil["data"]
print(data_casos_brasil.iloc[0])

regiao                                 Brasil
estado                                    NaN
municipio                                 NaN
coduf                                      76
codmun                                    NaN
codRegiaoSaude                            NaN
nomeRegiaoSaude                           NaN
data                      2020-07-29 00:00:00
semanaEpi                                  31
populacaoTCU2019                  210147125.0
casosAcumulado                        2552265
casosNovos                              69074
obitosAcumulado                         90134
obitosNovos                              1595
Recuperadosnovos                    1786753.0
emAcompanhamentoNovos                675378.0
interior/metropolitana                    NaN
Name: 155, dtype: object


In [9]:
# A maior quantidade de novos obitos no Brasil
maximo_obitos_novos_brasil = data_brasil["obitosNovos"].max()
data_obitos_brasil = data_brasil[data_brasil["obitosNovos"] == maximo_obitos_novos_brasil]
maximo_obitos_brasil = data_obitos_brasil["obitosNovos"]
date_obitos_brasil = data_obitos_brasil["data"]
print(data_obitos_brasil.iloc[0])

regiao                                 Brasil
estado                                    NaN
municipio                                 NaN
coduf                                      76
codmun                                    NaN
codRegiaoSaude                            NaN
nomeRegiaoSaude                           NaN
data                      2020-07-29 00:00:00
semanaEpi                                  31
populacaoTCU2019                  210147125.0
casosAcumulado                        2552265
casosNovos                              69074
obitosAcumulado                         90134
obitosNovos                              1595
Recuperadosnovos                    1786753.0
emAcompanhamentoNovos                675378.0
interior/metropolitana                    NaN
Name: 155, dtype: object


In [10]:
# Ver casos e obitos de todos os meses
relatorio_brasil = relatorio_meses(data_brasil, False, False)

In [11]:
print("Meses ordenados com o mês ordenado")
print(relatorio_brasil)

Meses ordenados com o mês ordenado
     casosNovos  obitosNovos  variacaoDeCasos  infectados  variacaoDeObitos  \
mes                                                                           
2             2            0             0.00         NaN              0.00   
3          5715          201        285650.00         NaN          20100.00   
4         79663         5700          1293.93     43544.0           2735.82   
5        428820        23413           438.29    278331.0            310.75   
6        887841        30280           107.04    552407.0             29.33   
7       1260444        32881            41.97    725959.0              8.59   
8       1245787        28906            -1.16    689157.0            -12.09   
9        902663        22571           -27.54    486607.0            -21.92   
10       724670        15932           -19.72    402823.0            -29.41   
11       484559         8729           -33.13    429449.0            -45.21   

     casosAcumul

In [12]:
print("Ordenado com base nos obitos")
print(relatorio_brasil.sort_values("obitosNovos", ascending=False).iloc[:, [0,1,2,3,4,5]])

Ordenado com base nos obitos
     casosNovos  obitosNovos  variacaoDeCasos  infectados  variacaoDeObitos  \
mes                                                                           
7       1260444        32881            41.97    725959.0              8.59   
6        887841        30280           107.04    552407.0             29.33   
8       1245787        28906            -1.16    689157.0            -12.09   
5        428820        23413           438.29    278331.0            310.75   
9        902663        22571           -27.54    486607.0            -21.92   
10       724670        15932           -19.72    402823.0            -29.41   
11       484559         8729           -33.13    429449.0            -45.21   
4         79663         5700          1293.93     43544.0           2735.82   
3          5715          201        285650.00         NaN          20100.00   
2             2            0             0.00         NaN              0.00   

     casosAcumulados  

In [13]:
print("Ordenado com base nos casos")
print(relatorio_brasil.sort_values("casosNovos", ascending=False).iloc[:, [0,1,2,3,4,5]])


Ordenado com base nos casos
     casosNovos  obitosNovos  variacaoDeCasos  infectados  variacaoDeObitos  \
mes                                                                           
7       1260444        32881            41.97    725959.0              8.59   
8       1245787        28906            -1.16    689157.0            -12.09   
9        902663        22571           -27.54    486607.0            -21.92   
6        887841        30280           107.04    552407.0             29.33   
10       724670        15932           -19.72    402823.0            -29.41   
11       484559         8729           -33.13    429449.0            -45.21   
5        428820        23413           438.29    278331.0            310.75   
4         79663         5700          1293.93     43544.0           2735.82   
3          5715          201        285650.00         NaN          20100.00   
2             2            0             0.00         NaN              0.00   

     casosAcumulados  


In [14]:
relatorio_brasil.to_csv("output/relatorios_brasil.csv" )

# Lendo dados pela região

In [15]:
print(data[data["regiao"] == "Nordeste"]["estado"].unique())

['MA' 'PI' 'CE' 'RN' 'PB' 'PE' 'AL' 'SE' 'BA']


In [16]:
aleatoria = data[data["regiao"] != "Brasil"].iloc[2710]
print(aleatoria)


regiao                               Nordeste
estado                                     RN
municipio                                 NaN
coduf                                      24
codmun                                    NaN
codRegiaoSaude                            NaN
nomeRegiaoSaude                           NaN
data                      2020-03-06 00:00:00
semanaEpi                                  10
populacaoTCU2019                    3506853.0
casosAcumulado                              0
casosNovos                                  0
obitosAcumulado                             0
obitosNovos                                 0
Recuperadosnovos                          NaN
emAcompanhamentoNovos                     NaN
interior/metropolitana                    NaN
Name: 2980, dtype: object


In [17]:
print(aleatoria["municipio"])

nan


In [18]:
regioes = data["regiao"].unique()[1:]

relatorios_regioes = {}

for regiao in regioes:
  data_regiao = data[data["regiao"] == regiao]
  data_regiao = data_regiao[data_regiao["municipio"].isna()]
  relatorio = relatorio_meses(data_regiao, True, relatorio_brasil)
  relatorios_regioes[regiao] = relatorio

C:\Users\Vitor\AppData\Local\Temp\ipykernel_12916\3502954567.py:47: RuntimeWarning: invalid value encountered in scalar divide
  comparacao_nacional_obitos = obitos / relatorio_mes["obitosNovos"].sum()


In [19]:
# Norte
print(relatorios_regioes["Norte"])

     casosNovos  obitosNovos  variacaoDeCasos  variacaoDeObitos  \
mes                                                               
2             0            0             0.00              0.00   
3           294            4         29400.00            400.00   
4         10482          708          3465.31          17600.00   
5         97019         4979           825.58            603.25   
6        154875         3836            59.63            -22.96   
7        145433         2323            -6.10            -39.44   
8        129687         1626           -10.83            -30.00   
9         93804         1570           -27.67             -3.44   
10        71964          932           -23.28            -40.64   
11        43537          523           -39.50            -43.88   

     casosAcumulados  obitosAcumulados  Casos por 100 mil habitantes  \
mes                                                                    
2                  0                 0             

In [20]:
# Nordest
print(relatorios_regioes["Nordeste"])

     casosNovos  obitosNovos  variacaoDeCasos  variacaoDeObitos  \
mes                                                               
2             0            0             0.00              0.00   
3           943           23         94300.00           2300.00   
4         23675         1518          2410.60           6500.00   
5        155680         7344           557.57            383.79   
6        301891        10436            93.92             42.10   
7        381418         9380            26.34            -10.12   
8        287974         6527           -24.50            -30.42   
9        178795         4026           -37.91            -38.32   
10       157041         2985           -12.17            -25.86   
11        88146         1527           -43.87            -48.84   

     casosAcumulados  obitosAcumulados  Casos por 100 mil habitantes  \
mes                                                                    
2                  0                 0             

In [21]:
# Centro-oeste
print(relatorios_regioes["Centro-Oeste"])

     casosNovos  obitosNovos  variacaoDeCasos  variacaoDeObitos  \
mes                                                               
2             0            0             0.00              0.00   
3           470            5         47000.00            500.00   
4          2229           76           374.26           1420.00   
5         14743          297           561.42            290.79   
6         79869         1363           441.74            358.92   
7        153723         3579            92.47            162.58   
8        183797         3908            19.56              9.19   
9        159947         3408           -12.98            -12.79   
10        99160         2190           -38.00            -35.74   
11        45957          970           -53.65            -55.71   

     casosAcumulados  obitosAcumulados  Casos por 100 mil habitantes  \
mes                                                                    
2                  0                 0             

In [22]:
# Sul
print(relatorios_regioes["Sul"])

     casosNovos  obitosNovos  variacaoDeCasos  variacaoDeObitos  \
mes                                                               
2             0            0             0.00              0.00   
3           680            9         68000.00            900.00   
4          4334          171           537.35           1800.00   
5         18088          369           317.35            115.79   
6         53788         1068           197.37            189.43   
7        152600         3303           183.71            209.27   
8        212855         4104            39.49             24.25   
9        151428         3085           -28.86            -24.83   
10       135411         2070           -10.58            -32.90   
11       142899         1587             5.53            -23.33   

     casosAcumulados  obitosAcumulados  Casos por 100 mil habitantes  \
mes                                                                    
2                  0                 0             

In [23]:
# Sudeste
print(relatorios_regioes["Sudeste"])

     casosNovos  obitosNovos  variacaoDeCasos  variacaoDeObitos  \
mes                                                               
2             2            0             0.00              0.00   
3          3462          162        173000.00          16200.00   
4         39004         3232          1026.63           1895.06   
5        145369        10448           272.70            223.27   
6        299013        13640           105.69             30.55   
7        432735        14349            44.72              5.20   
8        438057        12810             1.23            -10.73   
9        322756        10457           -26.32            -18.37   
10       264320         7796           -18.11            -25.45   
11       167051         4133           -36.80            -46.99   

     casosAcumulados  obitosAcumulados  Casos por 100 mil habitantes  \
mes                                                                    
2                  2                 0             

In [24]:
with pd.ExcelWriter("output/relatorio-regioes.xlsx") as writer:
    for key in relatorios_regioes:
      df = relatorios_regioes[key]
      df.to_excel(writer, sheet_name=key, index=False)